In [3]:
# LOAD PACKAGES

import matplotlib.pyplot as plt
import numpy as np
from numpy import asarray
import rioxarray as rxr
import geopandas as gpd
import xarray as xr
import pandas as pd
import glob
import os
import fnmatch
import math
from math import e
from osgeo import gdal 
print("packages loaded")

packages loaded


In [4]:
# SET DIRECTORIES

# Set basin directory
dnbr_basins = '/mnt/nfs/lss/meerdink/home/skzebarth/masters/output/'
print(sorted(os.listdir(dnbr_basins)))

# Set x_value directory
x_values = '/mnt/nfs/lss/meerdink/home/skzebarth/masters/basin_data/x_values.csv'
print(x_values)

# Set output directory
out = '/mnt/nfs/lss/meerdink/home/skzebarth/masters/basin_data/basin_pixel_values/'

# Create dataframe 
df = pd.read_csv(x_values, delimiter = ',')
print(df)

['12042_dnbr.tif', '12057_dnbr.tif', '12518_dnbr.tif', '12524_dnbr.tif', '12527_dnbr.tif', '12616_dnbr.tif', '12860_dnbr.tif', '12997_dnbr.tif', '13281_dnbr.tif', '17346_dnbr.tif', '17347_dnbr.tif', '17987_dnbr.tif', '19165_dnbr.tif', '19167_dnbr.tif', '19381_dnbr.tif', '19384_dnbr.tif', '19450_dnbr.tif', '19451_dnbr.tif', '19510_dnbr.tif', '19512_dnbr.tif', '19629_dnbr.tif', '19632_dnbr.tif', '19647_dnbr.tif', '19649_dnbr.tif', '19777_dnbr.tif', '20029_dnbr.tif', '20031_dnbr.tif', '20253_dnbr.tif', '20254_dnbr.tif', '20303_dnbr.tif', '20308_dnbr.tif', '20384_dnbr.tif', '20386_dnbr.tif', '20499_dnbr.tif', '20504_dnbr.tif', '20650_dnbr.tif', '20729_dnbr.tif', '20734_dnbr.tif', '21009_dnbr.tif', '21078_dnbr.tif', '21083_dnbr.tif', '21224_dnbr.tif', '21500_dnbr.tif', '21513_dnbr.tif', '21634_dnbr.tif', '22075_dnbr.tif', '22668_dnbr.tif']
/mnt/nfs/lss/meerdink/home/skzebarth/masters/basin_data/x_values.csv
    basin_id        x1        x2        x3
0      12042  0.197747  0.233633  0.13000

In [5]:
# GRAB PIXEL VALUE FROM EACH BASIN AND OUTPUT TO .CSV

# Iterate over files
for filename in sorted(os.listdir(dnbr_basins)):
    f = os.path.join(dnbr_basins, filename)
    if filename.endswith('.tif'):
        
        # Set empty list for .tif pixel array value
        pixarr = []
        
        #read in tif file
        basin = rxr.open_rasterio(f, masked=True)
        #print(basin)
        
        # Flatten the array, drop nan values, and append array values to list
        bflat = basin.values.flatten()
        #print(bflat)
        bflatnan = bflat[~np.isnan(bflat)]
        #print(bflatnan)
        pixarr.extend(bflatnan.tolist())
        #print(pixarr)
        
        # Grab file name for file output
        b_name = os.path.splitext(filename)[0]
        #print(b_name)
        
        # Create dataframe for pixel values, and output results to .csv
        pixel_df = pd.DataFrame(pixarr)
        pixel_df.columns = [b_name]
        print(pixel_df)
        
        pixel_df.to_csv(out + b_name +'_pv.csv', index=False)
print('Pixel values generated for basins')

      12042_dnbr
0       0.056069
1      -0.029344
2      -0.037474
3      -0.009698
4      -0.034133
...          ...
1200    0.038861
1201    0.027971
1202    0.142189
1203    0.093935
1204    0.067534

[1205 rows x 1 columns]
     12057_dnbr
0      0.845138
1      0.817937
2      0.719570
3      0.564456
4      0.479313
..          ...
802    0.419584
803    0.305340
804    0.575390
805    0.542316
806    0.458048

[807 rows x 1 columns]
      12518_dnbr
0      -0.096199
1      -0.046858
2       0.046132
3       0.018387
4      -0.016895
...          ...
1000    0.134038
1001    0.155298
1002    0.178350
1003    0.167072
1004    0.174184

[1005 rows x 1 columns]
     12524_dnbr
0      0.005162
1      0.000534
2      0.022537
3      0.122197
4      0.028882
..          ...
819    0.306313
820    0.482807
821    0.581953
822    0.526366
823    0.558612

[824 rows x 1 columns]
      12527_dnbr
0       0.065593
1       0.030697
2       0.011950
3       0.151732
4       0.138141
...     

     22668_dnbr
0      0.534272
1      0.461569
2      0.547426
3      0.476108
4      0.480545
..          ...
243   -0.002328
244    0.009412
245    0.020358
246    0.059757
247    0.037913

[248 rows x 1 columns]
Pixel values generated for basins


In [6]:
# THOMAS FIRE PRELIMINARY HAZARD ASSESSMENT LINK FUNCTION CALCULATION

# Coefficients for Southern California
b = -3.63
b_1 = 0.41
b_2 = 0.67
b_3 = 0.7


# Rainfall for design storm (12mm, 16mm, 20mm, 24mm, 28mm 32mm, 36mm, 40mm)
## Come back at a later date and make this section iterative as well
#R = 3
#R = 4
#R = 5
R = 6
#R = 7
#R = 8
#R = 9
#R = 10

### HERE IS WHERE I AM HAVING PROBLEMS ###
row_count = 0
for i in range(len(df)):
    row_val = df.iloc[row_count]
    x_1 = row_val['x1']
    x2 = []
    #print(x2)
    x_2 = []
    #print(x_2)
    x_3 = row_val['x3']
    #print(row_val)
    print(x_1)
    print(x_3)

    for filename in sorted(os.listdir(out)):
        
        f = os.path.join(out, filename)
        if filename.endswith('_pv.csv'):
           pix = pd.read_csv(f)
           x2.extend(pix.values.tolist())
    #print(x2) 
    for sublist in x2:
        for item in sublist:
            x_2.append(item)
   
    print(x_2)
    
    x =  b + (b_1 * x_1 * R) + (b_2 * x_2 * R) + (b_3 * x_3 * R)
    row_count +=1
    #print(x)
    
# Link function set up - empty lists are used to append values from the dataframe calculations
    #link_x = []
    #Prob = []
    #for index, row in df.iterrows():
        #x_1 = row['x1']
        #print(x_1)
        #x_2 = f
        #print(x_2)
        #x_3 = row['x3']
        #print(x_3)
        
        #link_x.append(x)


0.197747007
0.129999995
[0.0560691505670547, -0.0293438732624053, -0.037474438548088, -0.0096978396177291, -0.0341329127550125, -0.0471537709236145, -0.0808605849742889, -0.0401607602834701, -0.0372474491596221, -0.077487826347351, 0.0387687683105468, -0.0558803975582122, -0.0890924036502838, -0.0929389595985412, 0.071644127368927, -0.0020756125450134, -0.0393727719783782, -0.0684108138084411, -0.0137164592742919, -0.1009243726730346, -0.0729606747627258, -0.0289949029684066, 0.0911267399787902, 0.0868907049298286, 0.048602819442749, -0.0305964648723602, -0.0110107064247131, 0.0136312246322631, -0.0128891468048095, 0.0197107195854187, -0.0252742767333984, 0.012007862329483, 0.0591019243001937, 0.1477379947900772, 0.1438399255275726, 0.1161092221736908, 0.1422978937625885, 0.0896267220377922, 0.0401517376303672, 0.0532541647553443, 0.1088497340679168, 0.1371006667613983, 0.0423597693443298, -0.0088436603546142, 0.0108197927474975, 0.0329478979110717, 0.0181110501289367, 0.09106773138046

TypeError: can't multiply sequence by non-int of type 'float'